In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine, text

from getpass import getpass

In [2]:
pwd = getpass()
USER = "root"
HOST = "localhost"  
BD = "sakila"

In [3]:
con_url = f"mysql+pymysql://{USER}:{pwd}@{HOST}/{BD}"

engine = create_engine(con_url)

In [5]:
def rentals_month(engine, month, year):
    query = f"""
    SELECT customer_id, rental_id, rental_date
    FROM rental
    WHERE MONTH(rental_date) = {month}
      AND YEAR(rental_date) = {year};
    """
    df = pd.read_sql(query, engine)
    return df

In [6]:
def rental_count_month(df, month, year):
    col_name = f"rentals_{str(month).zfill(2)}_{year}"
    count_df = (
        df.groupby("customer_id")
        .size()
        .reset_index(name=col_name)
    )
    return count_df

In [7]:
def compare_rentals(df1, df2):
    merged = pd.merge(df1, df2, on="customer_id", how="inner")
    month1_col = merged.columns[1]
    month2_col = merged.columns[2]
    merged["difference"] = merged[month2_col] - merged[month1_col]
    return merged

In [8]:
may_df = rentals_month(engine, 5, 2005)
june_df = rentals_month(engine, 6, 2005)

may_count_df = rental_count_month(may_df, 5, 2005)
june_count_df = rental_count_month(june_df, 6, 2005)

comparison_df = compare_rentals(may_count_df, june_count_df)

print(comparison_df.head())

   customer_id  rentals_05_2005  rentals_06_2005  difference
0            1                2                7           5
1            2                1                1           0
2            3                2                4           2
3            5                3                5           2
4            6                3                4           1
